In [1]:
import gzip
import pickle 
import mplhep as hep
hep.style.use("CMS")
import matplotlib
matplotlib.rc('font', size=15)
import numpy as np
import torch 
import dgl
import pandas as pd
import numpy as np
import plotly.express as px
import sys
sys.path.append("/afs/cern.ch/work/m/mgarciam/private/Tracking_wcoc/")
from sklearn.cluster import DBSCAN, HDBSCAN
from torch_scatter import scatter_max, scatter_add, scatter_mean
from scipy.optimize import linear_sum_assignment
import numpy as np
import matplotlib.pyplot as plt
import dgl

/afs/cern.ch/work/m/mgarciam/private/miniconda/miniconda3/envs/graphgps0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
dic = torch.load("/eos/user/m/mgarciam/EVAL_REPOS/Tracking_CLD/models/281124/graphs/3.pt", map_location='cpu')
# g= dic["graph"]
# pos_hits_xyz = g.ndata["pos_hits_xyz"]
# hit_type = g.ndata["hit_type"].view(-1, 1)
# vector = g.ndata["vector"]
# input_ = torch.cat((pos_hits_xyz, hit_type, vector), dim=1)
# betas = torch.sigmoid(g.ndata["model_output"][:,3])
# X = g.ndata["model_output"][:,0:3]

In [29]:
from src.layers.inference_oc_tracks import get_clustering
from src.layers.inference_oc_tracks import hfdb_obtain_labels, get_clustering
dic["graph"].ndata["coords"] = dic["graph"].ndata["model_output"][:,0:3]
betas  = torch.sigmoid(dic["graph"].ndata["beta"])
X  = dic["graph"].ndata["coords"]
db = HDBSCAN(min_cluster_size=8, min_samples=8, cluster_selection_epsilon=0.3).fit(X)
labels_hdb = db.labels_ +1
labels_hdb = np.reshape(labels_hdb, (-1))
labels_hdb = torch.Tensor(labels_hdb).long()
# labels = hfdb_obtain_labels(X, betas.device)
labels = labels_hdb
from src.layers.inference_oc_tracks import match_showers, generate_showers_data_frame

clustering1 = get_clustering(betas, X, tbeta=0.6, td=0.3)
map_from = list(np.unique(clustering1.detach().cpu()))
cluster_id = map(lambda x: map_from.index(x), clustering1)
clustering_ordered = (
    torch.Tensor(list(cluster_id)).long()
)
if torch.unique(clustering1)[0] != -1:
    clustering = clustering_ordered + 1
else:
    clustering = clustering_ordered

labels = clustering.view(-1).long()
model_output = torch.cat((dic["graph"].ndata["coords"], dic["graph"].ndata["beta"].view(-1,1)), dim=1)
# model_output = dic["graph"].ndata["model_output"]
particle_ids = torch.unique(dic["graph"].ndata["particle_number"])
shower_p_unique = torch.unique(labels)
shower_p_unique, row_ind, col_ind, i_m_w,i_m_w_unique, iou_matrix= match_showers(
    labels,
    dic,
    particle_ids,
    model_output,
    0,
    0,
    None,
)
df_event, number_of_showers_total = generate_showers_data_frame(
                labels,
                dic,
                shower_p_unique,
                particle_ids,
                row_ind,
                col_ind,
                i_m_w,
                i_m_w_unique,
                number_of_showers_total=0,
                step=0,
                number_in_batch=0,
            )



In [33]:
from src.layers.inference_oc_tracks import evaluate_efficiency_tracks
df_batch_ct = evaluate_efficiency_tracks(
                dic["graph"],
                dic["graph"].ndata["ct_track_label"],
                dic["part_true"],
                0,
                0,
                0,
                path_save="",
                store=True,
                predict=False,
                ct=True,
                clustering_mode="clustering_normal",
                tau=False
            )

In [31]:
print(np.sum(~np.isnan(df_event.pred_showers_E)*(df_event.true_showers_E>0)),len(df_event[df_event.true_showers_E>0]))
num_fakes = np.sum(np.isnan(df_event.true_showers_E))
print(num_fakes)

29 32
0


In [39]:
print(np.sum(~np.isnan(df_batch_ct.pred_showers_E)*(df_batch_ct.true_showers_E>0)), len(df_batch_ct[df_batch_ct.true_showers_E>0]))
num_fakes = np.sum(np.isnan(df_batch_ct.true_showers_E))
print(num_fakes)

22 32
1


In [36]:
np.sum(~np.isnan(df_batch_ct.pred_showers_E)*(df_batch_ct.true_showers_E>0))

22

In [46]:

import pandas as pd
import plotly.express as px

from src.logger.plotting_tools import shuffle_truth_colors
g = dic["graph"]
id = 5
mask = (g.ndata['particle_number']!=-11)  #+(g.ndata['particle_number']==19)  #9, 17, 21, 22
tidx = labels[mask].view(-1,1) #(g.ndata['particle_number'][mask]).view(-1,1)  #labels[mask].view(-1,1)
#tidx =    1*(g.ndata['hit_link_modified'][mask].view(-1,1))+1
features = 1*(torch.ones_like(g.ndata["particle_number"][mask])).view(-1,1)
cellid= g.ndata['particle_number'][mask].view(-1,1)
X = g.ndata["coords"][mask][:,0:3] #[mask]
data = {
            "X":X[:, 0].view(-1, 1).detach().cpu().numpy(),
            "Y": X[:, 1].view(-1, 1).detach().cpu().numpy(),
            "Z": X[:, 2].view(-1, 1).detach().cpu().numpy(),
            "tIdx": tidx.view(-1, 1).detach().cpu().numpy(),
            "features": features.view(-1, 1).detach().cpu().numpy(),
            # "cellid":cellid.view(-1, 1).detach().cpu().numpy(),
        }
hoverdict = {}


df = pd.DataFrame(
np.concatenate([data[k] for k in data], axis=1),
columns=[k for k in data],
)
rdst = np.random.RandomState(1234567890)  # all the same
# shuffle_truth_colors(df, "tIdx", rdst)

hover_data = ["tIdx", "cellid"] #+ [k for k in hoverdict.keys()]
# if nidx is not None:
#     hover_data.append("av_same")
# fig = go.Figure()
fig = px.scatter_3d(
df,
x="X",
y="Y",
z="Z",
color="tIdx",
size="features",
# hover_data=hover_data,
template="simple_white",
range_color=[0,26],
# color_continuous_scale=px.colors.sequential.Turbo,
)
# fig.update_layout(
#     scene = dict(
#         xaxis = dict(visible=False),
#         yaxis = dict(visible=False),
#         zaxis =dict(visible=False)
#         )
#     )
fig.update_traces(marker=dict(line=dict(width=0)))
# fig.update_traces(marker_size = 2)
# mask_cdc= (g.ndata['hit_type'][mask] ==0)
# X_cdc = g.ndata["pos_hits_xyz"][mask][mask_cdc]
# vector_cdc = g.ndata["vector"][mask][mask_cdc]
# for i in range(0,torch.sum(g.ndata['hit_type'][mask] ==0)):
#     plot_single_arrow(fig,vector_cdc[i]*25,init_pt= X_cdc[i])
# fig.update_traces(marker=dict(line=dict(width=0)))
# fig.update_traces(marker_size = 2)
# fig.update_layout(
#     autosize=False,
#     width=800,
#     height=800,
# )
fig.show()

In [104]:
g.ndata["unique_id"][g.ndata["particle_number"]==21][6]

tensor(131072.)

In [89]:
def decode_id(id):
    # Extract calohit_col_index
    calohit_col_index = (id >> 16) & 0xFF
    
    # Extract side
    side = (id >> 8) & 0xFF
    
    # Extract layer
    layer = id & 0xFF
    
    return calohit_col_index, side, layer

In [103]:
print(decode_id(131072))
# print(decode_id(2))
# print(decode_id(4))
# print(decode_id(5))

# print(decode_id(131072))

# print(decode_id(327424))

(2, 0, 0)


In [77]:

# import pandas as pd
# import plotly.express as px

# from src.logger.plotting_tools import shuffle_truth_colors
# g = dgl.unbatch(dic["graph"])[0]
# mask = (g.ndata['particle_number'] ==6) #+(g.ndata['particle_number'] ==13)+(g.ndata['particle_number'] ==20) 
# tidx =  labels[mask].view(-1,1)
# #tidx =    1*(g.ndata['hit_link_modified'][mask].view(-1,1))+1
# features = 0.05* (torch.ones_like(g.ndata['particle_number'][mask].view(-1,1)))
# cellid= g.ndata['particle_number'][mask].view(-1,1)
# X = g.ndata["pos_hits_xyz"][mask] #[mask]
# data = {
#             "X":X[:, 0].view(-1, 1).detach().cpu().numpy(),
#             "Y": X[:, 1].view(-1, 1).detach().cpu().numpy(),
#             "Z": X[:, 2].view(-1, 1).detach().cpu().numpy(),
#             "tIdx": tidx.view(-1, 1).detach().cpu().numpy(),
#             "features": features.view(-1, 1).detach().cpu().numpy(),
#             "cellid":cellid.view(-1, 1).detach().cpu().numpy(),
#         }
# hoverdict = {}


# df = pd.DataFrame(
# np.concatenate([data[k] for k in data], axis=1),
# columns=[k for k in data],
# )
# rdst = np.random.RandomState(1234567890)  # all the same
# # shuffle_truth_colors(df, "tIdx", rdst)

# hover_data = ["tIdx", "cellid"] #+ [k for k in hoverdict.keys()]
# # if nidx is not None:
# #     hover_data.append("av_same")
# # fig = go.Figure()
# fig = px.scatter_3d(
# df,
# x="X",
# y="Y",
# z="Z",
# color="tIdx",
# size="features",
# hover_data=hover_data,
# template="simple_white",
# range_color=[0,40],
# color_continuous_scale=px.colors.sequential.Turbo,
# )
# # fig.update_layout(
# #     scene = dict(
# #         xaxis = dict(visible=False),
# #         yaxis = dict(visible=False),
# #         zaxis =dict(visible=False)
# #         )
# #     )
# fig.update_traces(marker=dict(line=dict(width=0)))
# fig.update_traces(marker_size = 2)


In [78]:
def match_showers(
    labels,
    dic,
    particle_ids,
    model_output,
    local_rank,
    i,
    path_save,
):
    iou_threshold = 0.1
    print("particle_ids", particle_ids)
    shower_p_unique = torch.unique(labels)
    print("shower_p_unique", shower_p_unique)
    if torch.sum(labels == 0) == 0:
        # if there is no zero it adds one
        shower_p_unique = torch.cat(
            (
                torch.Tensor([0]).to(shower_p_unique.device).view(-1),
                shower_p_unique.view(-1),
            ),
            dim=0,
        )
    # all hits weight the same
    e_hits = torch.ones_like(labels)
    i_m, i_m_w = obtain_intersection_matrix(
        shower_p_unique, particle_ids, labels, dic, e_hits
    )
    i_m = i_m.to(model_output.device)
    i_m_w = i_m_w.to(model_output.device)
    u_m = obtain_union_matrix(shower_p_unique, particle_ids, labels, dic)
    u_m = u_m.to(model_output.device)
    iou_matrix = i_m / u_m
    # taking from index 1 here excludes 0 the noise track from the predicted tracks
    if torch.sum(particle_ids == 0) > 0:
        # removing also the MC particle corresponding to noise 
        iou_matrix_num = (
            torch.transpose(iou_matrix[1:,1:], 1, 0).clone().detach().cpu().numpy()
        )
    else:
        iou_matrix_num = (
            torch.transpose(iou_matrix[1:, :], 1, 0).clone().detach().cpu().numpy()
        )
    iou_matrix_num[iou_matrix_num < iou_threshold] = 0
    row_ind, col_ind = linear_sum_assignment(-iou_matrix_num)
    # if torch.sum(particle_ids == 0) > 0:
    #     row_ind = row_ind + 1
    # next three lines remove solutions where there is a shower that is not associated and iou it's zero (or less than threshold)
    mask_matching_matrix = iou_matrix_num[row_ind, col_ind] > 0
    row_ind = row_ind[mask_matching_matrix]
    col_ind = col_ind[mask_matching_matrix]
    
    if torch.sum(particle_ids == 0) > 0:
        row_ind = row_ind + 1
    # if i == 0 and local_rank == 0:
    #     if path_save is not None:
    #         image_path = path_save + "/example_1_clustering.png"
    #         plot_iou_matrix(iou_matrix, image_path)
    # row_ind are particles that are matched and col_ind the ind of preds they are matched to
    return shower_p_unique, row_ind, col_ind, i_m_w, iou_matrix